### Below code explains how you define an Agent, Task, Tools and Crew objects and Kickoff to get the results from the LLM

In [1]:
# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Import the LLM class from crewai
from crewai import LLM

# Instantiate a language model using Gemini 2.0 Flash with a low temperature (more deterministic)
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.1
)

# Optional test call to check if the model works correctly
llm.call("Who invented transcendental meditation.")

'Transcendental Meditation (TM) was popularized by **Maharishi Mahesh Yogi**.\n'

In [3]:
# Define a tool to replace jargon and internal abbreviations in emails
# This tool will help in making emails more understandable by replacing jargon with specific terms

from crewai.tools import BaseTool

class ReplaceJargonsTool(BaseTool):
    name: str = "Jargon replacement tool"
    description : str = "Replaces jargon with more specific terms. "

    def _run(self, email: str) -> str:
        replacements = {
            "PRX": "Project Phoenix (internal AI revamp project)",
            "TAS": "technical architecture stack",
            "DBX": "client database cluster",
            "SDS": "Smart Data Syncer",
            "SYNCBOT": "internal standup assistant bot",
            "WIP": "in progress",
            "POC": "proof of concept",
            "ping": "reach out"
        }
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")

        return "\n".join(suggestions) if suggestions else "No jargon or internal abbreviations detected."

jt = ReplaceJargonsTool()

original_email = """
looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.
"""

jt.run(original_email)

Using Tool: Jargon replacement tool


"Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'\nConsider replacing 'TAS' with 'technical architecture stack'\nConsider replacing 'SDS' with 'Smart Data Syncer'\nConsider replacing 'SYNCBOT' with 'internal standup assistant bot'\nConsider replacing 'ping' with 'reach out'"

In [4]:
# Import required classes to build agents, tasks, and a crew
from crewai import Agent, Task, Crew

# Create an agent designed to improve email communication
email_assistant = Agent(
    role="Email Assistant Agent",  # Role of the agent
    goal="Improve emails and make them sound professional and clear",  # The agent's goal
    backstory="A highly experienced communication expert skilled in professional email writing",  # Background context
    tools=[jt],  # Tools available to the agent, including the jargon replacement tool
    verbose=True,  # Enables detailed logging
    llm=llm  # The LLM this agent will use
)

# Define a task to polish and professionalize the email
email_task = Task(
    description=f"""Take the following rough email and rewrite it into a professional and polished version.
    Expand abbreviations:
    '''{original_email}'''""",  # The content and instructions for the task
    agent=email_assistant,  # Assigning the email assistant agent to the task
    expected_output="A professional written email with proper formatting and content.",
)

# Group the agent and task into a crew for execution
crew = Crew(
    agents=[email_assistant],  # List of agents involved
    tasks=[email_task],        # List of tasks to be performed
    verbose=True               # Enables detailed execution logs
)

# Execute the crew's tasks and store the result
result = crew.kickoff()

# Output the result of the rewritten email
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ce14c540-b61e-4dd0-8d02-adfb5821c4f3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Task: Take the following rough email and rewrite it into a professional and polished version.                  │
│      Expand abbreviations:                                                                                      │
│      '''                                                                                                        │
│  looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.                      │
│  Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.                                          │
│  '''                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

e:\crewai-course\.venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Thought: The email needs to be more professional by expanding abbreviations and using clear language. I will   │
│  use the Jargon replacement tool to achieve this.                                                               │
│                                                                                                                 │
│  Using Tool: Jargon replacement tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"email\": \"looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday. Let'  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'                                   │
│  Consider replacing 'TAS' with 'technical architecture stack'                                                   │
│  Consider replacing 'SDS' with 'Smart Data Syncer'                                                              │
│  Consider replacing 'SYNCBOT' with 'internal standup assistant bot'                                             │
│  Consider replacing 'ping' with 'reach out'                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Project Updates and SDS Integration Timeline                                                          │
│                                                                                                                 │
│  Hi Team,                                                                                                       │
│                                                                                                                 │
│  I'm including Priya in this email thread. Updates on the technical architecture stack and Project Phoenix      │
│  (internal AI revamp project) are available in the presentation deck. The estimated time of arrival for Smart   │
│  Data Syncer integration is Friday.                                                                             │
│                                                                                                                 │
│  Let's schedule a meeting tomorrow, pending availability of the internal standup assistant bot 😄. Please       │
│  reach out to me if you encounter any impediments.                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a6722509-af3c-43fa-ba22-59d209f04e96                                                                     │
│  Agent: Email Assistant Agent                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ce14c540-b61e-4dd0-8d02-adfb5821c4f3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Project Updates and SDS Integration Timeline                                            │
│                                                                                                                 │
│  Hi Team,                                                                                                       │
│                                                                                                                 │
│  I'm including Priya in this email thread. Updates on the technical architecture stack and Project Phoenix      │
│  (internal AI revamp project) are available in the presentation deck. The estimated time of arrival for Smart   │
│  Data Syncer integration is Friday.                                                                             │
│                                                                                                                 │
│  Let's schedule a meeting tomorrow, pending availability of the internal standup assistant bot 😄. Please       │
│  reach out to me if you encounter any impediments.                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Subject: Project Updates and SDS Integration Timeline

Hi Team,

I'm including Priya in this email thread. Updates on the technical architecture stack and Project Phoenix (internal AI revamp project) are available in the presentation deck. The estimated time of arrival for Smart Data Syncer integration is Friday.

Let's schedule a meeting tomorrow, pending availability of the internal standup assistant bot 😄. Please reach out to me if you encounter any impediments.


✅ Flow / Highlights of the Code

1. `Environment Setup:`

    - Loads .env variables using load_dotenv() to securely handle environment configurations if any.

2. `Model Initialization:`

    - Initializes a Gemini 2.0 Flash model via LLM() with a deterministic temperature setting of 0.1.

3. `Defining a Tool`

    - Defined a tool to replace the Jargons in the raw email text if present.

4. `Agent Creation:`

    - Creates an "Email Assistant Agent" trained to convert rough/informal emails into professional, polished formats.

5. `Task Definition:`

    - Defines a task instructing the agent to rewrite a sample informal email into a clear and professional message.

6. `Crew Formation:`

    - Combines the agent and task into a Crew, enabling task management and execution.

7. `Execution:`

    - Executes the task via crew.kickoff() and prints the improved email output.